## CNN으로 MNIST 분류하기
이번 챕터에서는 CNN으로 MNIST를 분류해보겠습니다.

#### 1. 모델 이해하기
우리가 만들 모델의 아키텍처를 이해해봅시다. 합성곱 신경망은 출처에 따라서 합성곱 층을 부르는 단위로 조금 다릅니다.

#### 1-1. 첫번째 표기방법
`합성곱(nn.Conv2d)` + `활성화 함수(nn.ReLU)`를 하나의 합성곱 층으로 보고, `맥스풀링(nn.MaxPool2d)`은 풀링 층으로 별도도 명명합니다.

#### 1-2. 두번째 표기방법
`합성곱(nn.Conv2d)` + `활성화 함수(nn.ReLU)` + `맥스풀링(nn.MaxPool2d)`을 하나의 합성곱 층으로 봅니다.

다시 말해 풀링도 하나의 층으로 보느냐, 안 보느냐의 문제인데 누가 옳고 틀리냐의 문제는 아니므로, 이번 챕터에서는 편의를 위해 맥스풀링까지도 포함하여 하나의 합성곱 층으로 판단하고 정리해보겠습니다. 다시 말해 두번째 표기 방법을 택하겠습니다.

모델의 아키텍처는 총 3개의 층으로 구성됩니다.

```py
# 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax
```

이를 직접 구현해보며 이해해봅시다

## 모델 구현하기
위의 3개의 층을 직접 구현해보겠습니다.

In [1]:
import torch
import torch.nn as nn


C:\Users\LUDOBICO\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


임의의 텐서를 만듭니다. 텐서의 크기는 1 x 1 x 28 x 28 입니다.

In [9]:
# 배치 크기 x 채널 x 높이 x 너비의 텐서를 선언
inputs = torch.Tensor(1,1,28,28)

print('텐서의 크기 {}'.format(inputs.shape))

텐서의 크기 torch.Size([1, 1, 28, 28])


#### 합성곱층과 풀링 선언하기
이제 첫번째 합성곱 층을 구현해봅시다. 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1입니다.

In [10]:
conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


이제 두번째 합성곱 층을 구현해봅시다. 32채널 짜리를 입력받아서 64채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1입니다.

In [11]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


이제 맥스풀링을 구현해봅시다. 정수 하나를 인자로 넣으면 `커널 사이즈`와 `스트라이드` 둘 다 해당값으로 지정됩니다.

In [12]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


## 구현체를 연결하여 모델 만들기
지금까지는 선언만한 것이고 아직 이들을 연결시키지는 않았습니다. 이들을 연결시켜서 모델을 완성시켜보겠습니다. 우선 입력을 첫번째 합성곱층을 통과시키고 합성곱층을 통과시킨 후의 텐서의 크기를 보겠습니다.

In [13]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


`32채널`의 `28너비`,`28높이`의 텐서가 되었습니다. 32가 나온 이유는 `conv1`의 out_channel로 32를 지정해주었기 때문입니다. 또한, 28 너비 28 높이가 된 이유는 패딩을 1폭으로 하고 3 x 3 커널을 사용하면 크기가 보존되기 때문입니다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 보겠습니다.

In [14]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


`32채널`의 `14너비`,`14높이`의 텐서가 되었습니다. 이제 이를 다시 두번째 합성곱층엥 통과시키고 통과한 후의 텐서의 크기를 보겠습니다.

In [15]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


`64채널`의 `14너비`,`14높이`의 텐서가 되었습니다. 64가 나온 이유는 `conv2`의 out_channel로 64를 지정해주었기 때문입니다. 또한, 14너비 14높이가 된 이유는 패딩을 1폭으로 하고 3 x 3 커널을 사용하면 크기가 보존되기 때문입니다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 보겠습니다.

In [16]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


이제 이 텐서를 펼치는 작업을 할 겁니다. 그런데 펼치기에 앞서 텐서의 n번째 차원을 접근하게 해주는 `.size(n)`에 대해서 배워보겠습니다. 현재 out의 크기는 1 x 64 x 7 x 7입니다. out의 첫번째 차원이 몇인지 출력해보겠습니다.

In [17]:
print('첫 번째 out사이즈: ', out.size(0))
print('두 번째 out사이즈: ', out.size(1))
print('세 번째 out사이즈: ', out.size(2))
print('네 번째 out사이즈: ', out.size(3))


첫 번째 out사이즈:  1
두 번째 out사이즈:  64
세 번째 out사이즈:  7
네 번째 out사이즈:  7


이제 이를 가지고 `.view()`를 사용하여 텐서를 펼치는 작업을 해보겠습니다.

In [18]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


배치 차원을 제외하고 모두 하나의 차원으로 통합된 것을 볼 수 있습니다. 이제 이에 대해서 `전결합층(Fully-Connected Layer)`을 통과시켜보겠습니다. 출력층으로 10개의 뉴런을 배치하여 10개의 차원의 텐서로 변환합니다.

In [19]:
fc = nn.Linear(3136, 10) # input_dim = 3136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])
